In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import tqdm
import pickle
import math
from collections import Counter

pd.set_option('display.max_columns', None)

In [ ]:
mobile = pd.read_csv('Z:\\event_mobile_ppl.tsv', sep='\t',
                    dtype={'account_id': str, 'content_section.1': str, 'download_link': str, 'role': str,
                         'get-legal-help': str, 'keywords': str, 'Outgoing_link': str, 'Language': str,
                         'legal_position': str}, 
                     parse_dates=['event_date'],
                     index_col='event_id')

In [ ]:
mobile_contsect = mobile[~mobile['content_section'].isna()].copy()

In [ ]:
mobile_contsect['page_meta'] = list(zip(*[mobile_contsect[c] for c in ['page_url', 'content_section', 'event_date']]))

In [ ]:
not_procedure = mobile_contsect[mobile_contsect.content_section != 'Procedure']

In [ ]:
event_date = not_procedure.groupby('person_id').agg({'page_meta': 
                                              lambda x: set(tuple((a[0], b[0])) for a in x for b in x 
                                                            if ((b[2] - a[2] > pd.Timedelta('24:00:00')) and
                                                               (a[1] != b[1])))})

In [ ]:
pair_causal_counts = Counter([x for a in event_date.page_meta for x in list(a)])
pair_causal_counts.most_common()

In [ ]:
not_procedure[not_procedure['page_url'] == 'https://www.illinoislegalaid.org/legal-information/responding-order-protection-case']

In [ ]:
mobile['page_date'] = list(zip(*[mobile[c] for c in ['page_url', 'event_date']]))

In [ ]:
event_date = mobile.groupby('person_id').agg({'page_date': 
        lambda x: [(b[1] - a[1])/pd.Timedelta('24:00:00') for a in x for b in x
            if ((a[0] == 'https://www.illinoislegalaid.org/legal-information/how-much-money-can-i-make-and-still-get-snap-benefits')
            and (b[0] == 'https://www.illinoislegalaid.org/legal-information/am-i-eligible-medicaid'))]})

In [ ]:
time_btw_visits = [x for a in event_date.page_date for x in list(a)]

In [ ]:
time_btw_visits

In [ ]:
event_date_sec = not_procedure.groupby('person_id').agg({'page_meta': 
        lambda x: [(b[2] - a[2])/pd.Timedelta('24:00:00') for a in x for b in x
            if ((a[0] == 'https://www.illinoislegalaid.org/legal-information/how-much-money-can-i-make-and-still-get-snap-benefits')
            and (b[0] == 'https://www.illinoislegalaid.org/legal-information/am-i-eligible-medicaid'))]})

In [ ]:
time_btw_visits_sec = [x for a in event_date_sec.page_meta for x in list(a)]

In [ ]:
time_btw_visits_sec

In [ ]:
plt.hist(time_btw_visits_sec, bins=20)